In [1]:
import os   
import pandas as pd
import plotly.graph_objects as go
import datetime
import plotly.express as px

path = 'output_data/'
path_decade = 'output_data_decade/'

path = path_decade

all_files = os.listdir(path)
#all_files = os.listdir(path_decade)
all_files.sort()


files_245 = []
files_585 = []
for file in all_files:
    if '$' in file:
        continue
    if '._' in file:
        continue
    if '.DS' in file:
        continue
    if '245' in file:
        files_245.append(file)
    if '585' in file:
        files_585.append(file)



In [2]:
print(all_files)

['.DS_Store', 'EC-Earth3_ssp245.csv_decade_values.xlsx', 'TaiESM1_ssp585.csv_decade_values.xlsx']


In [3]:
variable_to_plot = 'HeatWaveTemperature_decade_max_raw'

# Getting columns/variables to compare
temp = pd.read_excel(path+all_files[1], index_col = 0)
columns_to_compare = temp.columns

# Comparing and graphing the variables
for variable in columns_to_compare:
    df_var_245 = pd.DataFrame()
    # Reads the variables from all files for ssp 245
    for file in files_245:
        temp_df = pd.read_excel(path+file, index_col = 0)
        temp_df.index = pd.to_datetime(temp_df.index)
        df_var_245.index = temp_df.index
        name = file.split('_')
        df_var_245[name[0]+'_'+variable] = temp_df[variable]   
    # Setting up min, avg, and max  
    df_var_245['min'] = 0.0
    df_var_245['avg'] = 0.0
    df_var_245['max'] = 0.0
    for i in range(len(df_var_245.index)):
        df_var_245.iloc[i,-3] = (df_var_245.iloc[i,0:-3].min())
        df_var_245.iloc[i,-2] = (df_var_245.iloc[i,0:-3].mean())
        df_var_245.iloc[i,-1] = (df_var_245.iloc[i,0:-3].max())

    df_var_585 = pd.DataFrame()
    # Reads the variables from all files for ssp 585
    for file in files_585:
        temp_df = pd.read_excel(path+file, index_col = 0)
        temp_df.index = pd.to_datetime(temp_df.index)
        df_var_585.index = temp_df.index
        name = file.split('_')
        df_var_585[name[0]+'_'+variable] = temp_df[variable] 
    # Setting up min, avg, and max  
    df_var_585['min'] = 0.0
    df_var_585['avg'] = 0.0
    df_var_585['max'] = 0.0
    for i in range(len(df_var_585.index)):
        df_var_585.iloc[i,-3] = (df_var_585.iloc[i,0:-3].min())
        df_var_585.iloc[i,-2] = (df_var_585.iloc[i,0:-3].mean())
        df_var_585.iloc[i,-1] = (df_var_585.iloc[i,0:-3].max())

    # Adjusting x-axis to Jan 1st rather than Dec 31st
    x_length = len(df_var_245.index)
    x_temp = pd.date_range(start = '2010-01-01 00:00:00', end = '2050-01-01 00:00:00', periods = x_length)

    # putting line plot of all the data
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        #x = df_var_245.index,
        x = x_temp,
        y = df_var_245['min'],
        marker_color = 'cornflowerblue',
        marker_size = 8,
        mode = 'lines',
        showlegend= False,
        opacity = 0.5
    ))
    fig.add_trace(go.Scatter(
        #x = df_var_245.index,
        x = x_temp,
        y = df_var_245['avg'],
        marker_color = 'cornflowerblue',
        marker_size = 16,
        mode = 'lines',
        fill='tonexty',
        name = 'SSP 245',
        showlegend= True,
        opacity = 0.5
    ))
    fig.add_trace(go.Scatter(
        #x = df_var_245.index,
        x = x_temp,
        y = df_var_245['max'],
        marker_color = 'cornflowerblue',
        marker_size = 8,
        mode = 'lines',
        fill='tonexty',
        showlegend= False,
        opacity = 0.5
    ))

    fig.add_trace(go.Scatter(
        #x = df_var_245.index,
        x = x_temp,
        y = df_var_585['min'],
        marker_color = 'firebrick',
        marker_size = 8,
        mode = 'lines',
        showlegend= False,
        opacity = 0.5
    ))
    fig.add_trace(go.Scatter(
        #x = df_var_245.index,
        x = x_temp,
        y = df_var_585['avg'],
        marker_color = 'firebrick',
        marker_size = 16,
        mode = 'lines',
        fill='tonexty',
        name = 'SSP 585',
        showlegend= True,
        opacity = 0.5
    ))
    fig.add_trace(go.Scatter(
        #x = df_var_245.index,
        x = x_temp,
        y = df_var_585['max'],
        marker_color = 'firebrick',
        marker_size = 8,
        mode = 'lines',
        fill='tonexty',
        showlegend= False,
        opacity = 0.5
    ))

    fig.update_xaxes(title = 'Year', range = [datetime.datetime(year = 2010, month = 1, day = 1),datetime.datetime(year = 2050, month = 1, day = 1)])
    unit = ''
    if 'tas' in variable:
        unit = 'Temperature (°C)'
    if 'Temperature' in variable:
        unit = 'Temperature (°C)'
    if 'pr' in variable: 
        unit = 'Precipitation (in)'
    if 'Precipitation' in variable:
        unit = 'Precipitation (in)'
    if 'Speed' in variable:
        unit = 'Speed (MPH)'
    if 'speed' in variable: 
        unit = 'Speed (MPH)'
    if 'Duration' in variable:
        unit = 'Number of Days'
    if 'Days' in variable:
        unit = 'Number of Days'
    if 'Frequency' in variable:
        unit = 'Number of Occurrences per Year'
    fig.update_yaxes(title = unit)

    fig.update_layout(template = 'simple_white', title = variable)
    fig.write_image('output_plots_combined/'+variable+'.png')
    if variable == variable_to_plot:
        fig.show()

    # Making a box plot
    fig = go.Figure()
    for i in df_var_245.index:
        fig.add_trace(go.Box(
            y = df_var_245.loc[i,df_var_245.columns[0]:df_var_245.columns[-4]],
            boxpoints='all',
            name = i.year,
            boxmean='sd'
            ))

    unit = ''
    if 'tas' in variable:
        unit = 'Temperature (°C)'
    if 'Temperature' in variable:
        unit = 'Temperature (°C)'
    if 'pr' in variable: 
        unit = 'Precipitation (in)'
    if 'Precipitation' in variable:
        unit = 'Precipitation (in)'
    if 'Speed' in variable:
        unit = 'Speed (MPH)'
    if 'speed' in variable: 
        unit = 'Speed (MPH)'
    if 'Duration' in variable:
        unit = 'Number of Days'
    if 'Days' in variable:
        unit = 'Number of Days'
    if 'Frequency' in variable:
        unit = 'Number of Occurrences per Year'
    fig.update_yaxes(title = unit)

    fig.update_xaxes(title = 'Year')
    fig.update_layout(template = 'simple_white', title = variable+' 245')
    fig.write_image('output_plots_combined/'+variable+'_245_box.png')
    if variable == variable_to_plot:
        fig.show()

    fig = go.Figure()
    for i in df_var_585.index:
        fig.add_trace(go.Box(
            y = df_var_585.loc[i,df_var_585.columns[0]:df_var_585.columns[-4]],
            boxpoints='all',
            name = i.year,
            boxmean='sd'
            ))

    unit = ''
    if 'tas' in variable:
        unit = 'Temperature (°C)'
    if 'Temperature' in variable:
        unit = 'Temperature (°C)'
    if 'pr' in variable: 
        unit = 'Precipitation (in)'
    if 'Precipitation' in variable:
        unit = 'Precipitation (in)'
    if 'Speed' in variable:
        unit = 'Speed (MPH)'
    if 'speed' in variable: 
        unit = 'Speed (MPH)'
    if 'Duration' in variable:
        unit = 'Number of Days'
    if 'Days' in variable:
        unit = 'Number of Days'
    if 'Frequency' in variable:
        unit = 'Number of Occurrences per Year'
    fig.update_yaxes(title = unit)

    fig.update_xaxes(title = 'Year')
    fig.update_layout(template = 'simple_white', title = variable+' 585')
    fig.write_image('output_plots_combined/'+variable+'_585_box.png')
    if variable == variable_to_plot:
        fig.show()


    

NameError: name 'variable_to_plot' is not defined

In [ ]:
# Looking just at intenstiy
# Getting columns/variables to compare
temp = pd.read_excel(path+all_files[1], index_col = 0)
columns_to_compare = temp.columns

df_245_intensity = pd.DataFrame()
df_585_intensity = pd.DataFrame()

# Comparing and graphing the variables
for variable in columns_to_compare:
    df_var_245 = pd.DataFrame()
    # Reads the variables from all files for ssp 245
    for file in files_245:
        temp_df = pd.read_excel(path+file, index_col = 0)
        temp_df.index = pd.to_datetime(temp_df.index)
        df_var_245.index = temp_df.index
        name = file.split('_')
        df_var_245[name[0]] = temp_df[variable]   

    df_245_intensity.index = df_var_245.columns
    df_245_intensity[variable] = df_var_245.iloc[len(df_var_245.index)-1]

    df_var_585 = pd.DataFrame()
    # Reads the variables from all files for ssp 585
    for file in files_585:
        temp_df = pd.read_excel(path+file, index_col = 0)
        temp_df.index = pd.to_datetime(temp_df.index)
        df_var_585.index = temp_df.index
        name = file.split('_')
        df_var_585[name[0]] = temp_df[variable] 

    df_585_intensity.index = df_var_585.columns
    df_585_intensity[variable] = df_var_585.iloc[len(df_var_585.index)-1]


#df_245_intensity.drop(labels = ['Year'], axis = 1, inplace = True)
#df_585_intensity.drop(labels = ['Year'], axis = 1, inplace = True)
df_245_intensity.dropna(axis = 1, inplace = True)
df_585_intensity.dropna(axis = 1, inplace = True)


In [ ]:
'''for column in df_245_intensity.columns:
    column_mean = df_245_intensity[column].max()
    df_245_intensity[column] = df_245_intensity[column]/column_mean

for column in df_585_intensity.columns:
    column_mean = df_585_intensity[column].max()
    df_585_intensity[column] = df_585_intensity[column]/column_mean'''


In [ ]:
'''fig = px.imshow(df_245_intensity.loc[:, 'hurs':'tasmin'], color_continuous_scale='RdBu_r')
fig.update_layout(height = 1000, width = 1000)
fig.write_image('Model_intensity_245.png')
fig.show()

fig = px.imshow(df_585_intensity.loc[:, 'hurs':'tasmin'],color_continuous_scale='RdBu_r')
fig.update_layout(height = 1000, width = 1000)
fig.write_image('Model_intensity_245.png')
fig.show()'''

In [ ]:
fig = px.imshow(df_245_intensity.transpose(), color_continuous_scale='RdBu_r')
fig.update_layout(height = 1000, width = 1000)
fig.write_image('output_plots_combined/Model_intensity_245.png')
fig.show()

fig = px.imshow(df_585_intensity.transpose(),color_continuous_scale='RdBu_r')
fig.update_layout(height = 1000, width = 1000)
fig.write_image('output_plots_combined/Model_intensity_585.png')
fig.show()